# Pricing vanilla call option using t-student random numbers
@Author: Daniel Rodriguez Delgado 

@Date: 19/01/2019


This assingment is very similar to the last one, "Hands-on blackshcoles model" but with the difference t-student numbers are used instead of gaussian ones. As well, the equation that is as follows:

\begin{equation}
S\leftarrow S\cdot e^{(r_c-d_c)\Delta t} +\sigma \cdot S\sqrt{\Delta t}\cdot \frac{t_n}{\sqrt{n / (n-2)}}
\end{equation}


As the t-student distribution applies a variance adjusted to its degrees of freedom, it's needed to standarize the variance as shown in the equation above


## 1. Pseudo-random number generators

In [9]:
from math import *
import numpy as np
import pandas as pd
from scipy.stats import kurtosis, skew, pearsonr, norm, t
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'


### 1.1 Congruential

In [2]:
def congru(seed, m=((2**31)-1), a=16807, b=0):
        y=(a*seed+b)%m
        u=y/m
        return(u, y)

### 1.2 Box-Muller

In [3]:
def boxmuller(seed, mu=0, sigma=1):
    u1, seed=congru(seed)    #Uniform inputs
    u2, seed=congru(seed)
    
    x=sqrt(-2*log(u1))*cos(2*pi*u2)*sigma+mu  #Gaussian numbers generation
    y=sqrt(-2*log(u1))*sin(2*pi*u2)*sigma+mu
    
    antx=sqrt(-2*log(1-u1))*cos(2*pi*(1-u2))*sigma+mu  #Antithetic numbers generation
    anty=sqrt(-2*log(1-u1))*sin(2*pi*(1-u2))*sigma+mu
    return(x, y, u1, u2, antx, anty, seed)

### 1.3 Marsaglia

In [4]:
def marsaglia(seed, mu=0, sigma=1, mode='normal'):
    
    while True:
        if mode=='normal':
            u1, seed= congru(seed)
            u2, seed= congru(seed)
            v1=2*u1-1
            v2=2*u2-1
            R2=v1**2+v2**2
            if R2<=1:
                y=sqrt(-2*log(R2)/R2)
                x1=v1*y*sigma+mu
                x2=v2*y*sigma+mu
                break
        if mode=='antithetic':
            u1, seed= congru(seed)
            u2, seed= congru(seed)
            v1=2*(1-u1)-1
            v2=2*(1-u2)-1
            R2=v1**2+v2**2
            if R2<=1:
                y=sqrt(-2*log(R2)/R2)
                x1=v1*y*sigma+mu
                x2=v2*y*sigma+mu
                break
        
    return(x1,x2,seed)

### 1.4 Gamma

In [11]:
def gamma (p, a, seed):
    v=[]
    for i in range(p):
        u, seed= congru(seed)
        v.append(log(u))
    x=-sum(v)/a
    return(x, seed)

### 1.5 Chi-square

In [6]:
# Chi^2 distribution random generator
def chisq (n, seed):
    if (n%2==0):
        g, seed=gamma(int(n/2), 1/2, seed)
        return(g, seed)
    else:
        g, seed=gamma(int((n-1)/2), 1/2, seed)
        cos1, sen1, u1, u2, antx, anty, seed = boxmuller(seed)
        x=g+sen1**2
    return(x, seed)

### 1.6 T-Student

In [7]:
def tstudent (n, seed):
    cos1, sen1, u1, u2, antx, anty, seed=boxmuller(seed)
    xchi, seed=chisq(n, seed)
    x=sen1/sqrt(xchi/n)
    return(x, seed)

## 2. Random numbers

In this section, 1k random numbers according to t-student distribution.

### 2.1 T-student method

In [26]:
N=12011
n=6
res=[]
seed=123456789

for i in range(N):
    x, seed=tstudent(n,seed)
    res.append(x)    



In [27]:
display(np.mean(res),np.var(res))


0.004507142959627083

1.4490458996509283

## 3. Initial data

In [28]:
# Underlying price at the begining
S0=14
# Strike price
Str=14

# Yearly and continous return
ry=0.03
rc=log(1+ry)

# Yearly and continuos discount rate
dy=0
dc=log(1+dy)

# Volatility
sig=0.3
#Period
T=1

## 4. Pricing

Using dynamic simulation with $\Delta t =1/12$ and stochastic equation:
\begin{equation}
S\leftarrow S\cdot e^{(r_c-d_c)\Delta t} +\sigma \cdot S\sqrt{t}\cdot \frac{t_s}{\sqrt{n/(n-2)}}
\end{equation}

In [30]:
d=12
dt=T/d

tM=np.empty((0,12))
# Expected pay-off for 12 months and 1000 simulations
payoff12c=[]
payoff12p=[]

for index, iem in enumerate(range(1,1001)):
    tm=[]
    j=index+1
    for i in range(d):
        if (i<1):
            te=S0*np.exp((rc-dc)*dt)+sig*S0*np.sqrt(dt)* (res[(j-1)*d+i]/np.sqrt(n/(n-2)))
            tm.append(te)
        else:
            te=te*np.exp((rc-dc)*dt)+sig*te*np.sqrt(dt)* (res[(j-1)*d+i]/np.sqrt(n/(n-2)))
            tm.append(te)
    tm=np.array(tm)    
    tM=np.vstack((tM,tm))
    
    if (tm[d-1]>Str):
        payoff12c.append(tm[d-1]-Str)
        payoff12p.append(0)
    else:
        payoff12c.append(0)
        payoff12p.append(Str-tm[d-1])

# Pay-off 12 months metrics for call option
po12AVGc=np.mean(payoff12c)
po12VARc=np.var(payoff12c, ddof=1)
po12ACCc=t.ppf(1-0.05/2,len(tM))*np.sqrt(po12VARc/1000)

val12AVGc=po12AVGc*np.exp(-rc*T)
val12VARc=po12VARc*np.exp(-2*rc*T)
val12ACCc=t.ppf(1-0.05/2,len(tM))*np.sqrt(val12VARc/1000)

index=['mean', 'variance', 'accuracy']
metricsc=pd.DataFrame({
                      'Pay-off': np.array([po12AVGc, po12VARc, po12ACCc]),
                      'Value': np.array([val12AVGc, val12VARc, val12ACCc])
                    }, index=index)


# Pay-off 12 months metrics for put option
po12AVGp=np.mean(payoff12p)
po12VARp=np.var(payoff12p, ddof=1)
po12ACCp=t.ppf(1-0.05/2,len(tM))*np.sqrt(po12VARp/1000)

val12AVGp=po12AVGp*np.exp(-rc*T)
val12VARp=po12VARp*np.exp(-2*rc*T)
val12ACCp=t.ppf(1-0.05/2,len(tM))*np.sqrt(val12VARp/1000)

index=['mean', 'variance', 'accuracy']
metricsp=pd.DataFrame({
                      'Pay-off': np.array([po12AVGp, po12VARp, po12ACCp]),
                      'Value': np.array([val12AVGp, val12VARp, val12ACCp])
                    }, index=index)

print('')
print(color.BOLD + 'Metrics for call option:')
display( metricsc)
print('')
print(color.BOLD + 'Metrics for put option:')
display(metricsp)


Metrics for call option:


,Pay-off,Value
mean,1.895378,1.840173
variance,8.666691,8.169188
accuracy,0.182684,0.177363



Metrics for put option:


,Pay-off,Value
mean,1.440439,1.398484
variance,3.929038,3.703495
accuracy,0.123003,0.119421
